# Setup

In [20]:
%run notebook_setup.py

Now you can import modules from the project root: /bi/workspace/Projects/Forecast/forecast


In [31]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean

from src.config.dir_config import OUTPUT_PATH_DEMMAND_SUMMARY,OUTPUT_PATH_PROCESSED_WEEKLY_SALES

In [22]:
demand_summary = pd.read_parquet(OUTPUT_PATH_DEMMAND_SUMMARY)

# Functions

In [23]:
def filter_dataframe(df, excludes_stores, excludes_deptos, exclude_ecom_deptos=True):
    if exclude_ecom_deptos:
        deptos_ecom = df[df['nombre_depto'].str.contains('e-com', case=False)]['nombre_depto'].unique().tolist()
        excludes_deptos = excludes_deptos + deptos_ecom

    df = df.loc[~df['cod_sucursal'].isin(excludes_stores)]
    df = df.loc[~df['nombre_depto'].isin(excludes_deptos)]

    # Si 'nombre_depto' y 'cod_sucursal' son categorías, eliminamos categorías no usadas
    for col in ['nombre_depto']:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            df[col] = df[col].cat.remove_unused_categories()

    return df

In [24]:
def safe_trim_mean(x, proportion=0.1):
   # Eliminar NaN antes de calcular trim_mean
   clean_x = x.dropna()
   if len(clean_x) == 0:
       return np.nan
   return trim_mean(clean_x, proportion)

# Filters

In [25]:
excludes_deptos = ['Miscelaneos', 'Bolsas y bolsos']
excludes_stores = [707, 767]

In [26]:
demand_summary = filter_dataframe(demand_summary, excludes_stores, excludes_deptos)

In [27]:
print(demand_summary['cod_sucursal'].isin([707, 767]).sum())
print(demand_summary['nombre_depto'].isin(excludes_deptos).sum())

0
0


# Demand summary

In [28]:
demand_summary_history = demand_summary.query('ano_temporada in ["2024", "2023", "2022"]').copy()

## 1) By season

In [29]:
demand_summary_by_season = demand_summary.groupby(['nombre_temporada', 'ano_temporada'], observed=True).agg(
   count=('cod_producto', 'count'),
   products=('cod_producto', 'nunique'),
   skus=('cod_sku', 'nunique'),
   stores=('cod_sucursal', 'nunique'),
   on_season_weeks=('on_season_weeks', lambda x: safe_trim_mean(x)),
   aviable_inventory_weeks=('aviable_inventory_weeks', lambda x: safe_trim_mean(x)),
   sales_weeks=('sales_weeks', lambda x: safe_trim_mean(x)),
   stockout_weeks=('stockout_weeks', lambda x: safe_trim_mean(x)),
   mean_adi=('ADI', lambda x: safe_trim_mean(x)),
   mean_sales=('mean_sale', lambda x: safe_trim_mean(x)),
   mean_cv_sales=('CV_sales', lambda x: safe_trim_mean(x)),
   mean_inventory=('mean_inventory', lambda x: safe_trim_mean(x)),
   mean_cv_inventory=('CV_inventory', lambda x: safe_trim_mean(x)),
).round(1).reset_index()

## 2) By Store

In [32]:
demand_summary_by_store = demand_summary.groupby(['nombre_sucursal','nombre_temporada','ano_temporada'], observed=True).agg(
   count=('cod_producto', 'count'),
   products=('cod_producto', 'nunique'),
   skus=('cod_sku', 'nunique'),
   stores=('cod_sucursal', 'nunique'),
   on_season_weeks=('on_season_weeks', lambda x: safe_trim_mean(x)),
   aviable_inventory_weeks=('aviable_inventory_weeks', lambda x: safe_trim_mean(x)),
   sales_weeks=('sales_weeks', lambda x: safe_trim_mean(x)),
   stockout_weeks=('stockout_weeks', lambda x: safe_trim_mean(x)),
   mean_adi=('ADI', lambda x: safe_trim_mean(x)),
   mean_sales=('mean_sale', lambda x: safe_trim_mean(x)),
   mean_cv_sales=('CV_sales', lambda x: safe_trim_mean(x)),
   mean_inventory=('mean_inventory', lambda x: safe_trim_mean(x)),
   mean_cv_inventory=('CV_inventory', lambda x: safe_trim_mean(x)),
).round(1).reset_index()


## 3) By Category

In [34]:
demand_summary_by_category = demand_summary.groupby(['nombre_depto', 'nombre_linea','nombre_temporada','ano_temporada'], observed=True).agg(
   count=('cod_producto', 'count'),
   products=('cod_producto', 'nunique'),
   skus=('cod_sku', 'nunique'),
   stores=('cod_sucursal', 'nunique'),
   on_season_weeks=('on_season_weeks', lambda x: safe_trim_mean(x)),
   aviable_inventory_weeks=('aviable_inventory_weeks', lambda x: safe_trim_mean(x)),
   sales_weeks=('sales_weeks', lambda x: safe_trim_mean(x)),
   stockout_weeks=('stockout_weeks', lambda x: safe_trim_mean(x)),
   mean_adi=('ADI', lambda x: safe_trim_mean(x)),
   mean_sales=('mean_sale', lambda x: safe_trim_mean(x)),
   mean_cv_sales=('CV_sales', lambda x: safe_trim_mean(x)),
   mean_inventory=('mean_inventory', lambda x: safe_trim_mean(x)),
   mean_cv_inventory=('CV_inventory', lambda x: safe_trim_mean(x)),
).round(1).reset_index()

## 4) By size

In [35]:
demand_summary_by_category_size = demand_summary.groupby(['nombre_depto', 'nombre_linea','cod_talla','nom_talla','nombre_temporada','ano_temporada'], observed=True).agg(
   count=('cod_producto', 'count'),
   products=('cod_producto', 'nunique'),
   skus=('cod_sku', 'nunique'),
   stores=('cod_sucursal', 'nunique'),
   on_season_weeks=('on_season_weeks', lambda x: safe_trim_mean(x)),
   aviable_inventory_weeks=('aviable_inventory_weeks', lambda x: safe_trim_mean(x)),
   sales_weeks=('sales_weeks', lambda x: safe_trim_mean(x)),
   stockout_weeks=('stockout_weeks', lambda x: safe_trim_mean(x)),
   mean_adi=('ADI', lambda x: safe_trim_mean(x)),
   mean_sales=('mean_sale', lambda x: safe_trim_mean(x)),
   mean_cv_sales=('CV_sales', lambda x: safe_trim_mean(x)),
   mean_inventory=('mean_inventory', lambda x: safe_trim_mean(x)),
   mean_cv_inventory=('CV_inventory', lambda x: safe_trim_mean(x)),
).round(1).reset_index()

## 5) By All

In [36]:
demand_summary_all = demand_summary.query('nombre_depto in ["Juvenil mujer", "Jeans juv mujer"]').groupby(['nombre_temporada','ano_temporada','nombre_depto', 'nombre_linea','cod_talla','nom_talla','nombre_sucursal'], observed=True).agg(
   count=('cod_producto', 'count'),
   products=('cod_producto', 'nunique'),
   skus=('cod_sku', 'nunique'),
   stores=('cod_sucursal', 'nunique'),
   on_season_weeks=('on_season_weeks', lambda x: safe_trim_mean(x)),
   aviable_inventory_weeks=('aviable_inventory_weeks', lambda x: safe_trim_mean(x)),
   sales_weeks=('sales_weeks', lambda x: safe_trim_mean(x)),
   stockout_weeks=('stockout_weeks', lambda x: safe_trim_mean(x)),
   mean_adi=('ADI', lambda x: safe_trim_mean(x)),
   mean_sales=('mean_sale', lambda x: safe_trim_mean(x)),
   mean_cv_sales=('CV_sales', lambda x: safe_trim_mean(x)),
   mean_inventory=('mean_inventory', lambda x: safe_trim_mean(x)),
   mean_cv_inventory=('CV_inventory', lambda x: safe_trim_mean(x)),
).round(1).reset_index()

## 6) By store_week

# Sample

In [37]:
productos = [273139, 273139,631139]
stores = [32, 207]

In [38]:
demand_summary_sample = demand_summary[(demand_summary['cod_producto'].isin(productos)) & (demand_summary['cod_sucursal'].isin(stores))].reset_index(drop=True).copy()

# Export to Excel

In [39]:
with pd.ExcelWriter('../sandbox/demand_summary.xlsx') as writer:
    demand_summary_sample.to_excel(writer, sheet_name='demand_summary', index=False)
    demand_summary_by_season.to_excel(writer, sheet_name='demand_summary_by_season', index=False)
    demand_summary_by_store.to_excel(writer, sheet_name='demand_summary_by_store', index=False)
    demand_summary_by_category.to_excel(writer, sheet_name='demand_summary_by_category', index=False)
    demand_summary_by_category_size.to_excel(writer, sheet_name='demand_summary_by_category_size', index=False)
    demand_summary_all.to_excel(writer, sheet_name='demand_summary_all', index=False)